In [155]:
from datetime import datetime, timedelta
import yfinance as yf   #https://pypi.org/project/yfinance/


In [156]:
# turn off debug to get data
debug = False

In [157]:
%run merge-data.ipynb

<ipython-input-157-11ba52ca5917>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['company']=df.apply(lambda x: getCompanyName(x.sponsor_collaborators, x.lead_sponsor, x.collaborators), axis=1)
<ipython-input-157-11ba52ca5917>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['symbol']=df.company.apply(lambda x: findSymbol(x, ticker_all))


In [158]:
# temporary turn on debug
debug = True

In [159]:
# question related source data overview
if (debug):
    print('source data shape: ', df.shape)
    print('total unique symbols:', df.symbol.nunique())
    print('symbol list', df.symbol.unique())

source data shape:  (367, 70)
total unique symbols: 152
symbol list ['PLRX' 'PPD' 'NVS' 'BCRX' 'GILD' 'REGN' 'SNY' 'DGX' 'VNDA' 'ABBV' 'PFE'
 'IFRX' 'CANF' 'BLDE' 'INO' 'CAPR' 'TRHC' 'CDNA' 'IMAB' 'LLY' 'EIGR' 'AZN'
 'BHVN' 'BMY' 'KPTI' 'TBPH' 'HGEN' 'UNH' 'INCY' 'ALXN' 'BHC' 'CNTG' 'MRKR'
 'BLPH' 'LBPS' 'MEDP' 'MESO' 'ATHX' 'IQV' 'BNTX' 'NVAX' 'IPHA' 'INMB'
 'VSTM' 'JNJ' 'GSK' 'IMUX' 'AIM' 'ECOR' 'BGNE' 'IBRX' 'VERU' 'PSTI' 'WINT'
 'CMRX' 'CTSO' 'EXAS' 'AVIR' 'XAIR' 'KNSA' 'ALVR' 'EDSA' 'HOPE' 'APLS'
 'VTGN' 'CTIC' 'CLOV' 'PTNR' 'LIFE' 'KLDO' 'RDHL' 'PHAR' 'SNDX' 'TAK'
 'DVA' 'MNOV' 'FGEN' 'PHAS' 'PTCT' 'SRNE' 'ALT' 'DRRX' 'CVAC' 'AMPE'
 'ANGN' 'AMRN' 'CRVS' 'VAR' 'BMRA' 'HRTX' 'BPTS' 'GRFS' 'ARCT' 'EVLO'
 'HAE' 'RETA' 'ADPT' 'VIR' 'MMM' 'ARPO' 'FULC' 'SNN' 'CLBS' 'GLBL' 'VRNA'
 'RDY' 'JAZZ' 'SAGE' 'MREO' 'IONS' 'KMDA' 'OPK' 'GRTX' 'EW' 'ILMN' 'VXRT'
 'DFFN' 'HRC' 'RIGL' 'NGM' 'AMGN' 'AEMD' 'CFRX' 'NRBO' 'EQ' 'CLNN' 'HUGE'
 'CERT' 'ONCS' 'LGVN' 'NKTR' 'ICLR' 'PRTC' 'ABIO' 'ENLV' 'EBS'

In [201]:
df['start_date']=pd.to_datetime(df['start_date'])
df['first_posted']=pd.to_datetime(df['first_posted'])
df['last_update_posted']=pd.to_datetime(df['last_update_posted'])

#df['start_date'][df.symbol=='CFRX']=df['first_posted']
df['start_date'][pd.isna(df.start_date)]=df['first_posted']

<ipython-input-201-d0f904714a1d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['start_date'][pd.isna(df.start_date)]=df['first_posted']


In [202]:
# get unique symbol and max time period to download price data
symbols = df[['symbol','start_date','last_update_posted']].groupby('symbol').agg(
    mindate=('start_date','min'),
    maxdate=('last_update_posted','max')
)
symbols = symbols.reset_index()

In [205]:
#df[df.symbol=='CFRX'][['start_date', 'last_update_posted', 'completion_date', 'company']]

In [211]:
# functions to download symbol price during a period
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)
        
def yfdownload(symbol, startdate, enddate):
    try:
        df = yf.download(symbol,start=startdate, end=enddate, progress=False, auto_adjust=True)
    except: 
        df = pd.DataFrame()
    return df

def getSymbolPriceData(symbol, mindate, maxdate):    
    startdate = mindate - timedelta(days=14) # two week before
    enddate = maxdate + timedelta(weeks=4)  # one month after  
#     startdate = datetime.strptime(mindate, "%Y-%m-%d") - timedelta(days=7) # one week before
#     enddate = datetime.strptime(maxdate, "%Y-%m-%d") + timedelta(weeks=4)  # one month after
#     print('symbol:', symbol,' startdate:', startdate, ' enddate:', enddate)
    
    df = pd.DataFrame()
    # some company went public after clinical trial, so no price for some dates
    for single_date in daterange(startdate, enddate):
        #print(single_date.strftime("%Y-%m-%d"))
        df = yfdownload(symbol,single_date, enddate)
        #print('downloaded data shape:',df.shape)
        if (df.shape[0]>0):
            # add symbol back to the data frame
            df['symbol']=symbol
            break
    
    return df

In [209]:
# getSymbolPriceData testing:
if (debug):
    print(getSymbolPriceData('ABC','2020-02-10','2020-02-10'))

symbol: ABC  startdate: 2020-02-03 00:00:00  enddate: 2020-03-09 00:00:00
                 Open       High        Low      Close   Volume symbol
Date                                                                  
2020-02-03  83.776490  84.731814  83.094107  83.542526  1014600    ABC
2020-02-04  84.956037  86.895938  84.819562  85.872368  1498600    ABC
2020-02-05  86.886179  90.142100  86.886179  89.138031  1608200    ABC
2020-02-06  89.800918  91.399631  89.245268  90.151855  1246100    ABC
2020-02-07  91.536107  92.335464  89.449981  89.625450  1467200    ABC
2020-02-10  89.391483  91.809053  89.060046  91.243652  1371600    ABC
2020-02-11  91.955276  92.228226  90.385806  92.189232  1156100    ABC
2020-02-12  92.608419  95.045483  91.672588  93.368782  1315100    ABC
2020-02-13  92.937952  93.486263  92.448392  92.546303  1155600    ABC
2020-02-14  92.193820  92.761703  88.345883  92.389641  1602200    ABC
2020-02-18  92.321119  92.908589  91.195131  92.340698  1566000    ABC
202

In [212]:
# download all symbol price data and save for late use since we may hit the daily free download limits
allprice = pd.DataFrame()
for row in symbols.itertuples(name='ticker'):
    allprice = allprice.append(getSymbolPriceData(row.symbol, row.mindate, row.maxdate))

allprice.to_csv('allprice' + str(datetime.now())[-7:-1] + '.csv')

symbol: ABBV  startdate: 2020-03-18 00:00:00  enddate: 2021-04-12 00:00:00
symbol: ABIO  startdate: 2020-11-23 00:00:00  enddate: 2021-05-03 00:00:00
symbol: ADMP  startdate: 2021-01-14 00:00:00  enddate: 2021-03-26 00:00:00
symbol: ADPT  startdate: 2020-07-17 00:00:00  enddate: 2021-05-04 00:00:00
symbol: AEMD  startdate: 2020-10-08 00:00:00  enddate: 2021-02-11 00:00:00
symbol: AIM  startdate: 2020-04-23 00:00:00  enddate: 2021-03-24 00:00:00
symbol: ALT  startdate: 2020-06-08 00:00:00  enddate: 2021-03-24 00:00:00
symbol: ALVR  startdate: 2020-05-12 00:00:00  enddate: 2021-04-29 00:00:00
symbol: ALXN  startdate: 2020-04-07 00:00:00  enddate: 2021-04-14 00:00:00
symbol: AMGN  startdate: 2020-10-05 00:00:00  enddate: 2021-04-27 00:00:00
symbol: AMPE  startdate: 2020-06-18 00:00:00  enddate: 2021-05-07 00:00:00
symbol: AMRN  startdate: 2020-06-23 00:00:00  enddate: 2020-12-21 00:00:00
symbol: ANGN  startdate: 2020-06-23 00:00:00  enddate: 2020-09-01 00:00:00

1 Failed download:
- ANGN:

In [135]:
## errors from yf:
# - ANGN: Data doesn't exist for startDate = 1598760000, endDate = 1598932800 (2020-07-29  2020-08-31)
# - CERT: Data doesn't exist for startDate = 1604552400, endDate = 1607576400 (2020-11-06  2020-12-09)
# - GLBL: Data doesn't exist for startDate = 1612328400, endDate = 1615352400 (2021-02-04  2021-03-09)
# - LBPS: Data doesn't exist for startDate = 1592971200, endDate = 1595995200 (2020-06-25  2020-07-28)
# - LGVN: Data doesn't exist for startDate = 1604898000, endDate = 1607922000 (2020-11-10  2020-12-13)
# - VALN: Data doesn't exist for startDate = 1614661200, endDate = 1617681600 (2021-03-03  2021-04-05)


In [213]:
# find out more about symbols has no price
if (debug):
    print('all symbol price data shape:', allprice.shape)
    tickers = set(df.symbol.unique()) - set(allprice.symbol.unique())
    print('no price symbols: ', tickers)
    
    tickerlist = ''
    for ticker in tickers:
        tickerlist = tickerlist + ' ' + ticker
        #hist = ticker.history(period="max")
        #print(hist)
    
    #tickers = yf.Tickers("ANGN CERT GLBL LBPS LGVN VALN" ) #['BPTS', 'CVAC']
    tickers = yf.Tickers(tickerlist.strip()) 
    
    hist=tickers.tickers['ANGN'].history(period="max").head(1)
    # hist=tickers.tickers['BPTS'].history(period="max").head(1)
    # hist=tickers.tickers['CVAC'].history(period="max").head(1)
    # hist=tickers.tickers['LBPS'].history(period="max").head(1)
    print(hist)

all symbol price data shape: (24699, 7)
no price symbols:  {'CERT', 'ANGN', 'LBPS', 'VALN', 'LGVN', 'GLBL'}
                 Open       High   Low  Close   Volume  Dividends  \
Date                                                                
2021-02-05  17.620001  18.985001  16.5  17.02  1667100          0   

            Stock Splits  
Date                      
2021-02-05             0  
